If necessary, install gymnasium.

In [ ]:
# !pip install gymnasium

In [ ]:
import base64

import gymnasium
import torch
import tqdm
import matplotlib.pyplot as plt

If you modify this notebook, use the following cell to format code cells when you run them.

In [ ]:
# import jupyter_black

# jupyter_black.load(
#     lab=False,
#     line_length=120,
# )

# Markov decision processes

Reinforcement learning (RL) is the study of improving decision makers through experience. Decision makers are often called **agents**. RL is often performed in settings called **environments**. Examples of environments are board games, driving courses, and human tissue. It's useful to interpret these environments as Markovian, or having the [Markov property](https://en.wikipedia.org/wiki/Markov_property). In such environments, each state is independent of the history of the environment. This is maybe better illustrated with an example.

Consider making a chess playing program. Chess is a board game played on an $8 \times 8$ square grid of spaces. Each space can be empty or have one of many chess pieces.

Not Markovian:
> Suppose we assign an integer to each type of chess piece, differentiating black from white pieces, as well as having a number for an empty space. Since there are 6 types of pieces and 2 colors, we'll have $6 + 6 + 1 = 13$ numbers. We can then then express an arrangement of pieces (called a *position* in chess) as an $8 \times 8$ matrix of numbers from 0 to 12. We could make a chess playing agent that decides its next move based on this state, but in order to play well, it has to consider not only the current position, but also past positions. One move, *en passant*, can only be performed immediately after an opponent makes it available. Also, agents that expect to win will want to avoid a stalemate, which occurs if the same position occurs 3 times. Therefore, if the agent takes such history into account, this environment is not Markovian.

Markovian:
> In contrast, suppose we encode relevant history into our state representation. In addition to the $8 \times 8$ matrix, we also have a number that indicates the number of times this position has occurred, as well as a number indicating in which of the columns *en passant* is enabled. Then such a state would really have all the information a good chess agent would care about, without having to consider past states.

A Markov decision process (MDP) is a set of these things:
* A state space, as exemplified above.
* An action space, which is the set of actions the agent may take in any state.
* A transition probabilty function, which maps (state, action, next state) tuples to a scalar probability of transitioning to some next state by being in some state and taking some action. 
* A reward function, which gives the reward the agent gets if they were in some state, took some action, and transitioned to some next state. The reward need not depend on all or any of these.

# Q-Learning

Q-learning is RL using a certain Q function that maps a state and action to the sum of expected future rewards, assuming the agent acts optimally from the next state on. The "Q" originally didn't stand for anything; it was just a letter to use for a function that no one bothered giving a good name to. Some people call it a *quality* function, to go with the Q. Others call it an action value function, which I prefer.

Suppose you have a state space $S$, action space $A$, transition probability function $t$, and reward function $r$. For now, assume that $S$ and $A$ are finite. Given that the agent is in some state $s \in S$, and takes some action $a \in A$, the Q function gives you the immediate reward plus the sum of expected future rewards given optimal behavior thereafter. The Q function is defined recursively as

$q(s, a) = \sum_{s' \in S} t(s, a, s') \left ( r(s, a, s') + \gamma \max_{a' \in A}q(s', a') \right )$

I've used $s'$ for next states and $a'$ for next actions. In order to get the agent to eventually seek reward, we have to discount the future rewards using a discount factor $\gamma$ such that $0 < \gamma < 1$, so that the agent prefers sooner rewards over later ones. Otherwise, it would assume it will live forever and not care. The closer to 0 $\gamma$ is, the more short-sighted the agent will be.

Terminal states end the game. The sequence of states, actions, and rewards from a starting state to a terminal state is called an *episode*. For any terminal state $z$ and any $a \in A$, $q(z, a) = 0$.

Because of the recursive nature of the Q function, you might see that naively evaluating it for every state would be extremely inefficient. Better would be to first evaluate states that transition to a terminal state, then evaluate states that transition into those, but since an agent cannot predict where terminal states are, it can't do this.

Also, even if it knew where the terminal states were, it probably doesn't know the functions $t$ and $r$, so at first, it won't be able to predict the results of its actions.

However, it can estimate the Q function using Q learning. By estimating $q$, which incorporates the outputs of $r$, it will learn to choose the most rewarding actions from any state.

Because we have a finite state space and action space, we can use a table, shaped $|S| \times |A|$, as an estimation of $q$ by using natural numbers to identify states and actions, and indexing into the table. Let's call the estimation $\hat{q}$. Initially, this table is filled with zeros.

Then, with some probability $\epsilon$, sometimes called *temperature*, we let the agent take a random action, and observe the result. We use the result to nudge the elements in the $q$ table toward our observation at some learning rate $\alpha$. Initially, $\epsilon$ should be close to 1. As the agent gets more experience in the environment, it will want to take fewer random actions and more reward-seeking actions, so it's reasonable to gradually decrease $\epsilon$. Once the $\hat{q}$ table is good enough, the agent can behave optimally from any state $s$ by choosing the action $\mathrm{argmax}_{a \in A}\hat{q}(s, a)$.

<div>
<img src="data/q_learning.png" width="600"/>
</div>

## Try it

Fill in the `train` function in the following `QLearningAgent` class. A solution is at the bottom of this notebook.

In [ ]:
class QLearningAgent(object):
    def __init__(self, environment: str):
        """
        Make a reinforcement learning agent that learns a table of action values.

        :param environment: gymnasium environment name
        """
        self.environment = gymnasium.make(environment)
        self.action_values = torch.zeros(self.environment.observation_space.n, self.environment.action_space.n)
        self.temperature = 1

    def train(
        self,
        n_episodes: int,
        learning_rate: float = 1,
        discount_factor: float = 0.9,
        temperature_decay_factor: float = 0.95,
    ) -> list[float]:
        """
        Repeatedly update the action value table with estimated action values while exploring the environment.

        :returns: the return per episode
        """
        returns = []
        episode_lengths = []
        for i in (progress_bar := tqdm.tqdm(range(n_episodes))):
            state, _ = self.environment.reset()
            is_terminal = False
            return_ = 0
            episode_length = 0
            while not is_terminal:
                episode_length += 1
                if torch.rand(1) < self.temperature:
                    action = self.environment.action_space.sample()
                else:
                    action = ...  # TODO
                next_state, reward, is_terminal, _, _ = self.environment.step(action)
                self.action_values[state, action] += ...  # TODO
                return_ += reward
                state = next_state
            self.temperature *= temperature_decay_factor

            returns.append(return_)
            progress_bar.set_postfix_str(f"return: {return_}")
            
            episode_lengths.append(episode_length)

        print(episode_lengths)
        return returns

You will use Q-learning to solve the [Taxi](https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py) environment. The Taxi problem from ["Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition" (Dietterich, 1998)](https://arxiv.org/abs/cs/9905014). It is a 2-dimensional grid-based environment where the goal of the agent is to pick up a passenger at one location and drop them off at another.

The agent has 6 actions, numbered 0 through 5.

The agent doesn't know the following. It's just for your information.
> The 6 actions consist of 4 actions for movement, 1 for pickup, and 1 for drop-off. Transitions are deterministic. Attempting a pickup when there is no passenger at the location incurs a reward of -10. Trying to drop off a passenger outside the drop-off zone incurs a reward of −10. Dropping the passenger off at the correct destination provides the agent with a reward of 20. Otherwise, the agent incurs a reward of −1 per time step.

We refer to the sum of rewards obtained over an episode as the **return** (not to be confused with the output of a function). Let's say you've solved it when the mean return over 10 consecutive episodes is positive. A correctly implemented agent should usually solve it using the following hyperparameters.

In [ ]:
EPISODES = 200
LEARNING_RATE = 1
DISCOUNT_FACTOR = 0.9
TEMPERATURE_DECAY_FACTOR = 0.98

We will train the agent. Since this is a very simple environment, this will not take long. Don't bother using a GPU; you'll spend more time moving data than you'll save on compute.

In [ ]:
agent = QLearningAgent("Taxi-v3")
returns = agent.train(
    n_episodes=EPISODES,
    learning_rate=LEARNING_RATE,
    discount_factor=DISCOUNT_FACTOR,
    temperature_decay_factor=TEMPERATURE_DECAY_FACTOR,
)

We will then plot the rolling mean returns and the temperature per episode.

In [ ]:
rolling_mean_returns = [sum(returns[i - 10 : i]) / 10 for i in range(10, EPISODES)]
temperatures = [TEMPERATURE_DECAY_FACTOR**i for i in range(EPISODES - 10)]
solved_episode = None
for episode in range(10, EPISODES):
    if rolling_mean_returns[episode - 10] > 0:
        solved_episode = episode
        break

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(range(10, EPISODES), rolling_mean_returns, color="tab:blue")
ax2.plot(range(10, EPISODES), temperatures, color="tab:orange")
if solved_episode:
    ax1.plot(solved_episode, rolling_mean_returns[solved_episode - 10], marker="o", color="black")
    y_low, y_high = ax1.get_ylim()
    ax1.text(solved_episode, rolling_mean_returns[solved_episode - 10] - (y_high - y_low) * 0.1, "solved", ha="center")
else:
    print("Not solved.")

ax1.set_xlabel("episode")
ax1.set_ylabel("mean return over last 10 episodes", color="tab:blue")
ax2.set_ylabel("temperature", color="tab:orange")

Worth discussing with your friends:
> How would you relate the expected episode length and an appropriate discount factor?

> The term *temperature* comes from an analogy with metallurgy, in which metal is heated and cooled at some schedule to make it softer. A better name might be *adventurousness*. Changing the temperature during training is one way to balance exploration (trying new things) and exploitation (taking advantage of what you are familiar with). Why is it a bad idea to have a constant temperature of 1? How about a constant temperature of 0?

Uncomment the following cell to see a solution.

In [ ]:
# print(
#     base64.b64decode(
#         b"YWN0aW9uID0gdG9yY2guYXJnbWF4KHNlbGYuYWN0aW9uX3ZhbHVlc1tzdGF0ZV0pLml0ZW0oKQoKc2VsZi5hY3Rpb25fdmFsdWVzW3N0YXRlLCBhY3Rpb25dICs9IGxlYXJuaW5nX3JhdGUgKiAoCiAgICByZXdhcmQgKyBkaXNjb3VudF9mYWN0b3IgKiBtYXgoc2VsZi5hY3Rpb25fdmFsdWVzW25leHRfc3RhdGVdKSAtIHNlbGYuYWN0aW9uX3ZhbHVlc1tzdGF0ZV1bYWN0aW9uXQop"
#     ).decode()
# )